In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [2]:
# Step 3: Define Helper Functions
def extract_player_crops(frame, boxes):
    crops = []
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        crop = frame[y1:y2, x1:x2]
        crop = cv2.resize(crop, (64, 128))  # Normalize size
        crops.append(crop)
    return crops

def get_color_histogram(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, [16, 16, 16],
                        [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

def compute_similarity_matrix(feats1, feats2):
    return cosine_similarity(feats1, feats2)


In [3]:

def detect_players(video_path, model):
    cap = cv2.VideoCapture(video_path)
    features = []
    frames_to_sample = 30  # Use only a few frames for speed
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    for i in tqdm(range(frame_count)):
        ret, frame = cap.read()
        if not ret:
            break

        if i % (frame_count // frames_to_sample) != 0:
            continue

        results = model(frame)
        bboxes = []

        for r in results:
            for box in r.boxes:
                cls_id = int(box.cls.item())
                if cls_id == 0:  # Player class only
                    bboxes.append(box.xyxy.cpu().numpy()[0])
        
        crops = extract_player_crops(frame, bboxes)
        for crop in crops:
            feat = get_color_histogram(crop)
            features.append(feat)

    cap.release()
    return features


In [5]:

model = YOLO("best.pt")  # Update this with your local path


In [8]:
# Step 1: Install Dependencies
!pip install ultralytics opencv-python scikit-learn tqdm --quiet



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
# Step 6: Extract Features from Both Videos
features_broadcast = detect_players("broadcast.mp4", model)
features_tacticam = detect_players("tacticam.mp4", model)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
# Step 7: Compare Features and Match Players
sim_matrix = compute_similarity_matrix(features_tacticam, features_broadcast)

print("Player Mapping Results:")
mapping = np.argmax(sim_matrix, axis=1)
for i, m in enumerate(mapping):
    print(f"Tacticam Player {i} → Broadcast Player {m}")
